# Message Requests to Email

In [24]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging
import json
import re
import base64
import pandas as pd

# Email
from email.mime.text import MIMEText

# Google Sheets API
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

Note from Manan: reference this chatgpt log for reference:
https://chatgpt.com/share/680ac284-24c8-8007-9707-b53951623755

Next steps:
1. Link the google service account correctly to the email service account
2. Connect one row of the data with a created message and one message request
3. Send one test email to one of the team members
4. Verify the test email works, then generalize it to work for all team members
5. Then generalize the code to loop through multiple files and send out these emails in an iterative manner

In [15]:
import os

# Assuming your credentials are in a folder named 'config' in the current working directory
config_folder = os.path.join(os.getcwd(), 'config')
credentials_path = os.path.join(config_folder, 'credentials.json')

if not os.path.exists(credentials_path):
    print(f"Credentials file not found: {credentials_path}")
else:
    print(f"Credentials path found: {credentials_path}")

Credentials path found: /Users/mananbhargava/Documents/Workspaces/dan-cs-ed/remind/email/config/credentials.json


In [23]:
# CHECK THIS CODE

# Path to your service account key
SERVICE_ACCOUNT_FILE = credentials_path

# The email of the Workspace account you're sending from
SENDER = 'your-admin-email@yourdomain.edu'

# Gmail API scope
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

# Authenticate using service account with domain-wide delegation
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES
)

# Delegate to the actual user
delegated_credentials = credentials.with_subject(SENDER)

# Build the service
service = build('gmail', 'v1', credentials=delegated_credentials)

In [18]:
# Read one message_request file from the email

df = pd.read_csv('message_requests/message_requests_Project 3: 2048.csv')
df

,first name,last name,sid,email,assignment,message_requests
0,Oindree,Chatterjee,3030000000,oindree@berkeley.edu,Project 3: 2048,"Dear Oindree, your Project 3: 2048 assignment ..."
1,Sarvagya,Goyal,3030000000,sgoyal123@berkeley.edu,Project 3: 2048,"Dear Sarvagya, your Project 3: 2048 assignment..."
2,Rian,Puri,3030000000,rian.puri@berkeley.edu,Project 3: 2048,"Dear Rian, your Project 3: 2048 assignment is ..."
3,Ankita,Sundar,3030000000,ankitasun@berkeley.edu,Project 3: 2048,"Dear Ankita, your Project 3: 2048 assignment i..."


Steps:
1. Iterate through each row of the message requests dataframe
2. Select the value from the email column, value from the message_requests column
3. Send the email in the google email api

In [ ]:
def create_message(sender, to, subject, message_text):
    """Create a message for an email."""
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}

In [20]:
def send_message(service, sender, message):
    """Send an email message."""
    try:
        sent_message = service.users().messages().send(userId=sender, body=message).execute()
        print(f'Message Id: {sent_message["id"]}')
    except Exception as error:
        print(f'An error occurred: {error}')

In [19]:
# Loop through each row of the message requests dataframe
# Incorporate the send message into this

for index, row in df.iterrows():
    # Extract the email address from the row
    email = row['email']
    # Extract the message from the row
    message = row['message_requests']
    # Extract the name from the row
    name = row['first name']
    # Extract the assignment name for the subject line title
    assignment_name = row['assignment']
    
    # Print the extracted information
    print(f"Email: {email}")
    print(f"Message: {message}")
    print(f"Name: {name}")

Email: oindree@berkeley.edu
Message: Dear Oindree, your Project 3: 2048 assignment is missing and it is due in 5 days 00:00:00. Please submit it as soon as possible.
Name: Oindree
Email: sgoyal123@berkeley.edu
Message: Dear Sarvagya, your Project 3: 2048 assignment is missing and it is due in 5 days 00:00:00. Please submit it as soon as possible.
Name: Sarvagya
Email: rian.puri@berkeley.edu
Message: Dear Rian, your Project 3: 2048 assignment is missing and it is due in 5 days 00:00:00. Please submit it as soon as possible.
Name: Rian
Email: ankitasun@berkeley.edu
Message: Dear Ankita, your Project 3: 2048 assignment is missing and it is due in 5 days 00:00:00. Please submit it as soon as possible.
Name: Ankita
